In [1]:
from pyspark import RDD, SparkContext
from pyspark.sql import SparkSession
from graphframes import *
from collections import defaultdict
import os
import sys
import math
import time


In [3]:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.2-spark3.1-s_2.12 pyspark-shell"

In [2]:
sc = SparkContext("local", "HW4").getOrCreate()
sc.setLogLevel("WARN")
spark = SparkSession.builder.appName('hw4').getOrCreate()

In [3]:
input = "ub_sample_data.csv"
filter_threshold = 7
output = "output2.txt"
commie = "output2.txt"

In [4]:
to_csv_RDD = sc.textFile(input).map(lambda line: line.split(','))
header = to_csv_RDD.first()
to_csv_RDD = to_csv_RDD.filter(lambda data: data != header)

In [5]:
user_biz_RDD = to_csv_RDD.map(lambda x: (x[0], x[1])).groupByKey().mapValues(lambda x: set(x))

In [48]:
all_pairs = user_biz_RDD.cartesian(user_biz_RDD).filter(lambda x: x[0] != x[1])  #Not filtering allows undirected
filtered_pairs = all_pairs.map(lambda x: ((x[0][0], x[1][0]), len(x[0][1].intersection(x[1][1])))).filter(
    lambda x: x[1] >= filter_threshold)

In [49]:
edges = filtered_pairs.map(lambda x: x[0])

In [50]:
vertices = filtered_pairs.flatMap(lambda x: x[0]).distinct()

In [51]:
network = edges.groupByKey().mapValues(set).collectAsMap()

In [10]:
expected_edges = edges.collect()

In [36]:
len(expected_edges)

996

In [11]:
class Node:
    def __init__(self, id):
        self.id = id
        self.height = 0
        self.node_credits = 1
        self.edge_credits = 0
        self.parents = {}
        self.children = {}

In [43]:
def betweeness_score(network):

    scores = defaultdict(float)
    for user in network.keys():

        # Shortest Paths
        cur_level = 0
        start_node = Node(user)
        bfs_tree = defaultdict(set)
        visited = {}
        bfs_tree[cur_level].add(user)
        visited[user] = start_node
        bfs_q = [start_node]

        while bfs_q:
            cur_node = bfs_q.pop(0)
            for adjacent_node in network[cur_node.id]:

                if adjacent_node in visited.keys():
                    adjacent_node = visited[adjacent_node]
                    if adjacent_node.height > visited[cur_node.id].height:
                        adjacent_node.node_credits += visited[cur_node.id].node_credits
                        visited[cur_node.id].children[adjacent_node] = adjacent_node
                        adjacent_node.parents[cur_node.id] = visited[cur_node.id]

                elif adjacent_node not in visited.keys():
                    new_Node = Node(adjacent_node)
                    new_Node.height = 1 + visited[cur_node.id].height
                    new_Node.parents[cur_node.id] = visited[cur_node.id]
                    new_Node.node_credits = visited[cur_node.id].node_credits
                    bfs_tree[visited[cur_node.id].height + 1].add(new_Node)
                    visited[cur_node.id].children[adjacent_node] = new_Node
                    visited[adjacent_node] = new_Node
                    bfs_q.append(new_Node)

        max_depth = max(bfs_tree.keys())


        # Calculating Betweenness
        for i in range(max_depth, 0, -1):
            for node in bfs_tree[i]:
                edge_score = 1 + node.edge_credits
                for id, pnode in node.parents.items():
                    r = (edge_score / node.node_credits)
                    edge_contribution = r *pnode.node_credits
                    pnode.edge_credits += edge_contribution
                    edge = frozenset({node.id, id})
                    scores[edge] += edge_contribution

    for score in scores:
        scores[score] /= 2
    return scores

In [46]:
ordered_output = []
for pair, score in b.items():
    ordered_pair = tuple((sorted(tuple(pair)), score))
    ordered_output.append(ordered_pair)

In [47]:
with open(output, "w+") as f:
    for pair, score in sorted(ordered_output, key= lambda x:(x[1],x[0][1]), reverse=True):
        f.write("(" + str(pair[0]) + "," + str(pair[1]) + ")," + str(score) +"\n")


In [52]:
degree_map = {}
two_m = 0
for node in network:
    degree_map[node] = len(network[node])
    two_m+=len(network[node])

In [53]:
degree_map

{'39FT2Ui8KUXwmUt6hnwy-g': 23,
 '0FVcoJko1kfZCrJRfssfIA': 53,
 '_6Zg4ukwS0kst9UtkfVw3w': 2,
 'JM0GL6Dx4EuZ1mprLk5Gyg': 28,
 'bSUS0YcvS7UelmHvCzNWBA': 31,
 'LcCRMIDz1JgshpPGYfLDcA': 9,
 'lJFBgSAccsMGwIjfD7LMeQ': 2,
 'DKolrsBSwMTpTJL22dqJRQ': 12,
 'sdLns7062kz3Ur_b8wgeYw': 1,
 'MtdSCXtmrSxj_uZOJ5ZycQ': 12,
 '23o7tyUGlC6FCDVcyqLeFA': 1,
 'CLbpPUqP6XpeAfoqScGaJQ': 1,
 '_VTEyUzzH92X3w-IpGaXVA': 26,
 'qtOCfMTrozmUSHWIcohc6Q': 4,
 'zBi_JWB5uUdVuz3JLoAxGQ': 17,
 'KLB3wIYUwKDPMbijIE92vg': 13,
 '1KQi8Ymatd4ySAd4fhSfaw': 3,
 '_Pn-EmWO-pFPFg81ZIEiDw': 8,
 'dTeSvET2SR5LDF_J07wJAQ': 12,
 'EiwxlbR8fb68lMgEXhcWKA': 4,
 '4pc_EyanaC3ARh0MZZyouA': 7,
 '2XYdguaaZ7dgi6fAlddujg': 2,
 'bE7Yd0jI_P6g27MWEKKalA': 17,
 'm1IVpXClMox1VGw5hO2LhQ': 4,
 'IuaAfrkirlfzY3f4PkgSmw': 4,
 'ay4M5J28kBUf0odOQct0BA': 13,
 'dzJDCQ5vubQBJTfYTEmcbg': 4,
 'mu4XvWvJOb3XpG1C_CHCWA': 2,
 'OoyQYSeYNyRVOmdO3tsxYA': 3,
 'PE8s8ACYABRNANI-T_WmzA': 1,
 'sBqCpEUn0qYdpSF4DbWlAQ': 5,
 '7RCz4Ln_FaTvNrdwe251Dg': 7,
 '1st2ltGKJ00ZcRsev-Ieew': 4

In [58]:
def get_modularity_score(network):


    community_list = []
    traversed = set()
    for node in network.keys():
        if node not in traversed:
            bfs_tree = {node}
            bfs_q = [node]
            while bfs_q:
                cur_node = bfs_q.pop(0)
                for adjacent_node in network[cur_node]:
                    if adjacent_node not in bfs_tree:
                        bfs_tree.add(adjacent_node)
                        bfs_q.append(adjacent_node)

            community_list.append(bfs_tree)
            traversed = traversed.union(bfs_tree)

    modularity_score = 0
    for c in community_list:
        for i in c:
            k_i = degree_map[i]
            for j in c:
                k_j = degree_map[j]
                #modularity_score -= float(k_i * k_j) / float(two_m)
                #if ((i, j)) in expected_edges or ((j,i)) in expected_edges:
                #    modularity_score += 1
                if ((i, j)) in expected_edges:
                    modularity_score+= 1 - k_i * k_j / two_m
                else:
                    modularity_score+= 0 - k_i * k_j / two_m

    modularity_score /= two_m

    return modularity_score, community_list

get_modularity_score(network)

(0.07852292704956468,
 [{'0FMte0z-repSVWSJ_BaQTg',
   '0FVcoJko1kfZCrJRfssfIA',
   '0KhRPd66BZGHCtsb9mGh_g',
   '0QREkWHGO8-Z_70qx1BIWw',
   '0gZ8E5tBWTEtGEZDuTzhzw',
   '1KQi8Ymatd4ySAd4fhSfaw',
   '1st2ltGKJ00ZcRsev-Ieew',
   '23o7tyUGlC6FCDVcyqLeFA',
   '2GUjO7NU88cPXpoffYCU8w',
   '2XYdguaaZ7dgi6fAlddujg',
   '2k8OVAPxlXHsA5X6EIoQpQ',
   '2quguRdKBzul3GpRi9e1mA',
   '2xVrxhQJUBmOyG4ML77XKw',
   '37HswRimgBEf7_US-c3CDA',
   '39FT2Ui8KUXwmUt6hnwy-g',
   '4ONcRRisDZkbV1cviA7nFw',
   '4PQhC-zTQ4ACEN0-r39JuQ',
   '4ZQq0ozRs-gXSz1z55iIDw',
   '4pc_EyanaC3ARh0MZZyouA',
   '5DgFmyjW6hkBtXtTMKl4tA',
   '5fQ9P6kbQM_E0dx8DL6JWA',
   '6YmRpoIuiq8I19Q8dHKTHw',
   '6xi9tBoZ6r_v41u_XFsSnA',
   '750rhwO7D_Cul7_GtO9Jsg',
   '79yaBDbLASfIdB-C2c8DzA',
   '7G8w2SnaC-qDVQ7_GqTxMg',
   '7RCz4Ln_FaTvNrdwe251Dg',
   '7Vfy39A_totC-w70qZi0MA',
   '8oYMqhC5fhqAK_yxRjE7dQ',
   '903YwVSoAKyzudc8LH_HMA',
   '97j2wkFU46OOgm6ErRAb7w',
   '9S52XHEyrvOv4OZxU6pCLw',
   '9SWtEX1k9AjRg93BAzMCpg',
   '9W73B44Iw8WslrTNB

In [30]:
def get_modularity_score(network):


    community_list = []
    traversed = set()
    for node in network.keys():
        if node not in traversed:
            bfs_tree = {node}
            bfs_q = [node]
            while bfs_q:
                cur_node = bfs_q.pop(0)
                for adjacent_node in network[cur_node]:
                    if adjacent_node not in bfs_tree:
                        bfs_tree.add(adjacent_node)
                        bfs_q.append(adjacent_node)

            community_list.append(bfs_tree)
            traversed = traversed.union(bfs_tree)

   # m_double = sum(node_degree.values()) # 2m, i.e. doulbe of num edges in original graph
    res = 0
    for community in community_list:
        for node_i in community:
            k_i = degree_map[node_i]
            for node_j in community:
                k_j = degree_map[node_j]
                res -= k_i * k_j / two_m
                if frozenset({node_i, node_j}) in expected_edges:
                    res += 1
    return (res / two_m), community_list

    return modularity_score, community_list

get_modularity_score(saved_network)

(-0.9214770729505822,
 [{'0FMte0z-repSVWSJ_BaQTg',
   '0FVcoJko1kfZCrJRfssfIA',
   '0KhRPd66BZGHCtsb9mGh_g',
   '0QREkWHGO8-Z_70qx1BIWw',
   '0gZ8E5tBWTEtGEZDuTzhzw',
   '1KQi8Ymatd4ySAd4fhSfaw',
   '1st2ltGKJ00ZcRsev-Ieew',
   '23o7tyUGlC6FCDVcyqLeFA',
   '2GUjO7NU88cPXpoffYCU8w',
   '2XYdguaaZ7dgi6fAlddujg',
   '2k8OVAPxlXHsA5X6EIoQpQ',
   '2quguRdKBzul3GpRi9e1mA',
   '2xVrxhQJUBmOyG4ML77XKw',
   '37HswRimgBEf7_US-c3CDA',
   '39FT2Ui8KUXwmUt6hnwy-g',
   '4ONcRRisDZkbV1cviA7nFw',
   '4PQhC-zTQ4ACEN0-r39JuQ',
   '4ZQq0ozRs-gXSz1z55iIDw',
   '4pc_EyanaC3ARh0MZZyouA',
   '5DgFmyjW6hkBtXtTMKl4tA',
   '5fQ9P6kbQM_E0dx8DL6JWA',
   '6YmRpoIuiq8I19Q8dHKTHw',
   '6xi9tBoZ6r_v41u_XFsSnA',
   '750rhwO7D_Cul7_GtO9Jsg',
   '79yaBDbLASfIdB-C2c8DzA',
   '7G8w2SnaC-qDVQ7_GqTxMg',
   '7RCz4Ln_FaTvNrdwe251Dg',
   '7Vfy39A_totC-w70qZi0MA',
   '8oYMqhC5fhqAK_yxRjE7dQ',
   '903YwVSoAKyzudc8LH_HMA',
   '97j2wkFU46OOgm6ErRAb7w',
   '9S52XHEyrvOv4OZxU6pCLw',
   '9SWtEX1k9AjRg93BAzMCpg',
   '9W73B44Iw8WslrTNB

In [27]:
get_modularity_score(saved_network)

TypeError: argument of type 'PipelinedRDD' is not iterable

In [18]:
saved_network = network.copy()

In [430]:
network = saved_network.copy()

In [23]:
network = saved_network.copy()
best_q, c_list = get_modularity_score(network)
cut_counter = len(expected_edges)
while (cut_counter>1):

    betweeness_scores = betweeness_score(network)
    betweeness_scores = sorted(betweeness_scores.items(), key=lambda x:x[1], reverse=True)
    max_btwness = betweeness_scores[0][1]
    cut_edges = []
    for e, val in betweeness_scores:
        if val == max_btwness:
            cut_edges.append(list(e))
    for e in cut_edges:
        cut_counter-=2
        print(cut_counter)
        node1 = e[0]
        node2 = e[1]
        network[node1] = network[node1]-{node2}
        network[node2] = network[node2]-{node1}
        print(best_q, cut_q)
    cut_q, cut_c_list = get_modularity_score(network)
    if best_q < cut_q:
        best_q = cut_q
        c_list = cut_c_list



994
-0.8429541459011597 -0.026910856276511737
992
-0.8429541459011597 -0.8429541459011597
990
-0.8429541459011597 -0.8429541459011597
988
-0.8429541459011597 -0.8429541459011597
986
0.05341849325010304 0.05341849325010304
984
0.05341849325010307 0.05341849325010307
982
0.05341849325010307 0.05341849325010307
980
0.05341849325010307 0.05341849325010307
978
0.05341849325010307 0.05341849325010307
976
0.05341849325010307 0.05341849325010307
974
0.05341849325010307 0.05341849325010307
972
0.05341849325010307 0.05341849325010307
970
0.2698867760197184 0.2698867760197184
968
0.32023677037465254 0.32023677037465254
966
0.32023677037465254 0.32023677037465254
964
0.32023677037465254 0.32023677037465254
962
0.3295753294301504 0.3295753294301504
960
0.3295753294301504 0.3295753294301504
958
0.3967839228399364 0.3967839228399364
956
0.3967839228399364 0.3967839228399364
954
0.3967839228399364 0.3967839228399364
952
0.41266269898870683 0.41266269898870683
950
0.41266269898870683 0.4126626989887068

KeyboardInterrupt: 

In [20]:
task2_output = []
for i in c_list:
    task2_output.append(sorted(i))

In [21]:
with open(commie, "w+") as f:
    for i in sorted(task2_output, key = lambda x:(len(x),x)):
        f.write("\'")
        f.write('\', \''.join(i))
        f.write("\'")
        f.write("\n")